## Load EPA GHG Major Emitters data from 2021 datasets (see https://www.epa.gov/ghgreporting/data-sets) for original sources

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have local copies rooted in the S3_BUCKET : s3://redhat-osc-physical-landing-647521352890/EPA/ghgp_data_parent_company_10_2021.xlsx and s3://redhat-osc-physical-landing-647521352890/EPA/ghgp_2020_data_summary_spreadsheets/

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials and Data Commons libraries

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

from osc_ingest_trino import *
import pyarrow as pa
import pyarrow.parquet as pq
import json
import io
import uuid

Create an S3 resource for the bucket holding source data

In [ ]:
import boto3
s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
# Create an S3 client.  We will user later when we write out data and metadata
s3_trino = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [ ]:
import trino

conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
ingest_uuid = str(uuid.uuid4())

custom_meta_key_fields = 'metafields'
custom_meta_key = 'metaset'

ingest_schema = 'epa_ghgrp'
cur.execute(f'create schema if not exists osc_datacommons_dev.ingest_schema')
cur.fetchall()

Time for the Pandas!

In [ ]:
import pandas as pd
import numpy as np
import re

For osc_datacommons_dev, a trino pipeline is a parquet data stored in the S3_DEV_BUCKET
It is a 5-step process to get there from a pandas dataframe

In [ ]:
def create_trino_pipeline (s3, schemaname, tablename, timestamp, df, meta_fields, meta_content):
    global ingest_uuid
    global custom_meta_key_fields, custom_meta_key
    
    # First convert dataframe to pyarrow for type conversion and basic metadata
    table = pa.Table.from_pandas(df)
    # Second, since pyarrow tables are immutable, create a new table with additional combined metadata
    if meta_fields or meta_content:
        meta_json_fields = json.dumps(meta_fields)
        meta_json = json.dumps(meta_content)
        existing_meta = table.schema.metadata
        combined_meta = {
            custom_meta_key_fields.encode(): meta_json_fields.encode(),
            custom_meta_key.encode(): meta_json.encode(),
            **existing_meta
        }
        table = table.replace_schema_metadata(combined_meta)
    # Third, convert table to parquet format (which cannot be written directly to s3)
    pq.write_table(table, f'/tmp/{schemaname}.{tablename}.{ingest_uuid}.{timestamp}.parquet')
    # df.to_parquet(f'/tmp/{schemaname}.{tablename}.{uuid}.parquet', index=False)
    # Fourth, put the parquet-ified data into our S3 bucket for trino.  We cannot compute parquet format directly to S3 but we can copy it once computed
    s3.upload_file(
        Bucket=os.environ['S3_DEV_BUCKET'],
        Key=f'trino/{schemaname}/{tablename}/{ingest_uuid}/{timestamp}/data.parquet',
        Filename=f'/tmp/{schemaname}.{tablename}.{ingest_uuid}.{timestamp}.parquet'
    )
    # Finally, create the trino table backed by our parquet files enhanced by our metadata
    cur.execute(f'drop table if exists osc_datacommons_dev.{schemaname}.{tablename}')
    print(f'dropping table: {tablename}')
    cur.fetchall()
    
    columnschema = create_table_schema_pairs(df)

    tabledef = f"""create table if not exists osc_datacommons_dev.{schemaname}.{tablename}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{os.environ['S3_DEV_BUCKET']}/trino/{schemaname}/{tablename}/{ingest_uuid}/{timestamp}'
)"""
    print(tabledef)

    # tables created externally may not show up immediately in cloud-beaver
    cur.execute(tabledef)
    cur.fetchall()

Load EPA GHGP data file using pandas *read_excel* and using *ingest_uuid* as the global UUID for this ingestion

In [ ]:
bObj = bucket.Object('EPA/ghgp_data_parent_company_10_2021.xlsx')
ghgp_bytes = io.BytesIO(bObj.get()['Body'].read())
timestamp = bObj.last_modified.isoformat()

custom_meta_content = { 'foo':'bar'}
custom_meta_fields = { 'GHGRP FACILITY ID':'GHGRP FACILITY ID'}

# The source data mistakenly codes ZIP as a number, which means ZIP codes like 02134 are coded as 2134

ghgp_parent = pd.read_excel(ghgp_bytes, sheet_name=None, dtype={'FRS ID (FACILITY)':'string', 'FACILITY NAICS CODE':'string', 'PARENT CO. PERCENT OWNERSHIP':'float32'}, parse_dates=['REPORTING YEAR'], date_parser=lambda x: pd.to_datetime(x, format='%Y'), engine='openpyxl')
for year in ghgp_parent.keys():
    df = ghgp_parent[year]
    df.loc[df['PARENT COMPANY NAME']=='INTERNATIONAL PAPER CO', ['PARENT CO. STREET ADDRESS']] = '6400 Poplar Ave.'
    df.loc[df['PARENT COMPANY NAME']=='Iowa Army Ammunition Plant', ['PARENT CO. STREET ADDRESS']] = '17571 DMC Highway 79'
    df.loc[df['FACILITY NAME']=='Avon Lake Power Plant', ['FACILITY ADDRESS']] = '16157 Co Rd 22'
    df.loc[df['FACILITY NAME']=='ENCANA OIL AND GAS USA - FORT LUPTON GAS PLANT', ['FACILITY ADDRESS']] = '33570 Lake Rd'
    df['FACILITY ZIP'] = df['FACILITY ZIP'].astype(str).str.zfill(5)
    df['PARENT CO. ZIP'] = df['PARENT CO. ZIP'].astype(str).str.zfill(5)
    # print(year + ' ' + str([x for x in ghgp_parent[year]['PARENT CO. STREET ADDRESS'].to_list() if x and re.match(r'^\d+$', str(x))]))
    
    gleif_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],f'mtiemann-GLEIF/ghgp-{year}-matches.csv')
    gleif_file.download_file(f'/tmp/ghgp-gleif.csv')
    gleif_df = pd.read_csv(f'/tmp/ghgp-gleif.csv', header=0, sep=',', dtype=str, engine='c')
    gleif_df.dropna(subset=['LEI'], inplace=True)
    gleif_dict = dict(zip(gleif_df['PARENT COMPANY NAME'], gleif_df.LEI))
    
    df['LEI'] = df['PARENT COMPANY NAME'].map(gleif_dict)
    print(f'year = {year}; non-NULL LEIs = {len(df[df.LEI.notnull()])}')
    
    cols = df.columns.tolist()
    ghgp_parent[year] = df[cols[0:2] + [cols[-1]] + cols[2:-1]]
    enforce_sql_column_names(ghgp_parent[year], inplace=True)

In [ ]:
# As a next step, load per-year data, which is more detailed
# i.e., bObj = bucket.Object(f'EPA/ghgp_2020_data_summary_spreadsheets/ghgp_data_{year}.xlsx')

bObj = bucket.Object(f'EPA/ghgp_2020_data_summary_spreadsheets/ghgp_data_by_year.xlsx')
ghgp_bytes = io.BytesIO(bObj.get()['Body'].read())
timestamp = max(timestamp, bObj.last_modified.isoformat())
    
ghg_data = pd.read_excel(ghgp_bytes, sheet_name=None, dtype={'FRS Id':'string'}, skiprows=3, engine='openpyxl')
for df_name, df in ghg_data.items():
    if df_name=='Industry Type':
        break
    if 'Zip Code' in df:
        df['Zip Code'] = df['Zip Code'].astype(str).str.zfill(5)
    elif 'Reported Zip Code' in df:
        df['Reported Zip Code'] = df['Reported Zip Code'].astype(str).str.zfill(5)
    df['Primary NAICS Code'] = df['Primary NAICS Code'].astype(str)
    df.replace('confidential', -1, inplace=True)
    if df_name=='CO2 Injection':
        # All info essentially confidential, but at least we know where facilities are
        ghg_data[df_name] = df
        continue
    df = df.convert_dtypes()
    # Some numeric info marked 'confidential' which will make for interesting NA handling...
    # df.info(verbose=True)
    ghg_data[df_name] = df

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [ ]:
tablename = 'parent_company'
create_trino_pipeline (s3_trino, ingest_schema, tablename, timestamp, pd.concat(ghgp_parent, ignore_index=True), custom_meta_fields, custom_meta_content)

Restore data and metadata

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3_trino.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key=f'trino/{ingest_schema}/{tablename}/{ingest_uuid}/{timestamp}/data.parquet'
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
restored_meta

In [ ]:
restored_meta_fields

### Melt the generation data into a more tidy format, dropping NA values

In [ ]:
for tablename, ghg_table in ghg_data.items():
    if tablename=='Industry Type':
        break
    print(f':{tablename}:')
    # Melt the data...
    if tablename in ['Gathering & Boosting', 'Transmission Pipelines', 'Geologic Sequestration of CO2']:
        ghg_value_vars = ghg_table.columns[-5:]
        ghg_id_vars = ghg_table.columns[:-5]
    elif tablename=='Suppliers':
        continue
    else:
        ghg_value_vars = ghg_table.columns[-10:]
        ghg_id_vars = ghg_table.columns[:-10]
    _, value_name = ghg_value_vars[0].split(' ', 1)
    ghg_melted_df = ghg_table.melt(ghg_id_vars, ghg_value_vars, var_name='year', value_name=value_name)
    ghg_melted_df.dropna(subset=[value_name],inplace=True)
    ghg_melted_df.year = ghg_melted_df.year.apply(lambda x: x.split(' ', 1)[0])
    ghg_melted_df['year'] = pd.to_datetime(ghg_melted_df['year'], format='%Y', utc=True)
    ghg_melted_df[value_name] = ghg_melted_df[value_name].astype('float64')
    tablename = sql_compliant_name(tablename)
    enforce_sql_column_names(ghg_melted_df, inplace=True)
    create_trino_pipeline (s3_trino, ingest_schema, tablename, timestamp, ghg_melted_df, custom_meta_fields, custom_meta_content)

In [ ]:
tablename = 'Suppliers'
ghg_table = ghg_data[tablename].copy()
ghg_table['Zip Code'] = ghg_table['Zip Code'].astype(str).str.zfill(5)

ghg_value_names = [':'.join(list(reversed(x.split(' ', 1)))) for x in ghg_table.columns[-70:]]
ghg_table = ghg_table.rename(columns=dict(zip(ghg_table.columns[-70:], ghg_value_names)))
ghg_id_vars = ghg_table.columns[:-70]
# print(ghg_id_vars)
# print(ghg_value_names)
stubnames = [x.split(':', 1)[0] for x in ghg_value_names[0:70:10]]
# print(stubnames)
suppliers_df = pd.wide_to_long(ghg_table, stubnames=stubnames, i=ghg_id_vars, j='year', sep=':')
# Take care to null out all the `confidential` data
for sn in stubnames:
    suppliers_df[sn] = suppliers_df[sn].astype('Float64')
suppliers_df.dropna(subset=stubnames, how='all', inplace=True)
suppliers_df.reset_index(inplace=True)
suppliers_df.loc[suppliers_df.year.notnull(), 'year'] = pd.to_datetime(suppliers_df.year, format='%Y', utc=True)

for sn in stubnames:
    new_stubname = ' '.join(sn.split(' ')[4:] + ['ghg'])
    suppliers_df.rename(columns={sn:new_stubname}, inplace=True)

tablename = sql_compliant_name(tablename)
enforce_sql_column_names(suppliers_df, inplace=True)
create_trino_pipeline (s3_trino, ingest_schema, tablename, timestamp, suppliers_df, custom_meta_fields, custom_meta_content)

In [ ]:
# Everything below here is speculative / in process of design

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*

In [ ]:
# Create metastore structure
metastore = {'catalog':'osc_datacommons_dev',
             'schema':'epa_ghgrp_md',
             'table':tablename,
             'metadata':custom_meta_content,
             'uuid':ingest_uuid}
# Create DataFrame
df_meta = pd.DataFrame(metastore)
# Print the output
df_meta

In [ ]:
cur.execute('show tables in osc_datacommons_dev.epa_ghgrp')
l = cur.fetchall()
totalrows = 0
for e in l:
    s = f'select count (*) from osc_datacommons_dev.epa_ghgrp.{e[0]}'
    print(s)
    cur.execute(s)
    rowcount = cur.fetchall()[0][0]
    totalrows += rowcount
    print(rowcount)
    
print(f'total rows = {totalrows}')

In [ ]:
cur.execute(f'select count (*), primary_naics_code from osc_datacommons_dev.epa_ghgrp.direct_emitters group by primary_naics_code order by count (*) desc')
cur.fetchall()

In [ ]:
cur.execute(f"select count (*) from osc_datacommons_dev.epa_ghgrp.parent_company where LEI is not null")
cur.fetchall()

In [ ]:
cur.execute(f"describe osc_datacommons_dev.epa_ghgrp.parent_company")
cur.fetchall()

In [ ]:
cur.execute(f"describe osc_datacommons_dev.epa_ghgrp.direct_emitters")
cur.fetchall()

In [ ]:
list(gleif_dict.items())[0]

In [ ]:
cur.execute(f"describe osc_datacommons_dev.sec_dera.sub")
cur.fetchall()

In [ ]:
cur.execute(f"describe osc_datacommons_dev.sec_dera.num")
cur.fetchall()

In [ ]:
cur.execute(f"describe osc_datacommons_dev.epa_ghgrp.direct_emitters")
cur.fetchall()

In [ ]:
cur.execute('select * from osc_datacommons_dev.epa_ghgrp.direct_emitters limit 1')
cur.fetchall()

In [ ]:
cur.execute(f"""select ghgrp_facility_id, osc_datacommons_dev.epa_ghgrp.parent_company.lei, parent_company_name, total_reported_direct_emissions, reporting_year
from osc_datacommons_dev.epa_ghgrp.parent_company, osc_datacommons_dev.epa_ghgrp.direct_emitters, osc_datacommons_dev.sec_dera.sub
where osc_datacommons_dev.epa_ghgrp.parent_company.lei is not null and reporting_year >= DATE('2020-01-01') and year=reporting_year
and osc_datacommons_dev.epa_ghgrp.parent_company.ghgrp_facility_id=osc_datacommons_dev.epa_ghgrp.direct_emitters.facility_id
and osc_datacommons_dev.epa_ghgrp.parent_company.lei=osc_datacommons_dev.sec_dera.sub.lei 
-- and osc_datacommons_dev.sec_dera.sub.adsh=osc_datacommons_dev.sec_dera.num.adsh
-- and osc_datacommons_dev.sec_dera.num.tag LIKE '%Revenue%'
-- and osc_datacommons_dev.sec_dera.num.ddate < DATE('2021-01-01')
-- and osc_datacommons_dev.sec_dera.num.ddate >= DATE('2020-09-30')
order by total_reported_direct_emissions desc
limit 10
""")
cur.fetchall()